# 7. Neuronales Netz für Regression

**KI1-Projekt 308** — California Housing Datensatz

**Schwerpunkt P5 (Kernaufgabe):** Neuronales Netz mit TensorFlow/Keras
für die Vorhersage von Hauspreisen. Vergleich mit linearer Regression.

Vorlage: Blatt 12 (TensorFlow Regression), Kapitel 8 Folien

> ⚠️ **Python-Version:** Dieses Notebook benötigt **Python 3.13**.  
> TensorFlow (≥ 2.18) ist **nicht** kompatibel mit Python 3.14+.  
> Venv erstellen mit: `python3.13 -m venv .venv`

In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.linear_model import LinearRegression

from utils.data import load_and_clean_data, get_train_test_split
from utils.evaluation import evaluate_predictions, add_result
from utils.plotting import plot_predicted_vs_actual, plot_residuals, save_fig

plt.rcParams['figure.dpi'] = 100
%matplotlib inline

print(f"TensorFlow Version: {tf.__version__}")

## 7.1 Daten laden (skaliert)

Neuronale Netze benötigen skalierte Eingabedaten für effizientes Training.

In [ ]:
df = load_and_clean_data()
X_train, X_test, y_train, y_test, scaler, feature_names = get_train_test_split(df, scaler='standard')

# Validierungssplit aus Trainingsdaten
val_split = int(0.8 * len(X_train))
X_val, y_val = X_train[val_split:], y_train[val_split:]
X_train_nn, y_train_nn = X_train[:val_split], y_train[:val_split]

print(f"Training:   {X_train_nn.shape}")
print(f"Validation: {X_val.shape}")
print(f"Test:       {X_test.shape}")
print(f"Features:   {feature_names}")

## 7.2 Referenz: Lineare Regression

Aufgabenstellung fordert explizit den Vergleich mit linearer Regression.

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

result_lr = evaluate_predictions(
    y_train, lr.predict(X_train),
    y_test, lr.predict(X_test),
    "Lineare Regression (Referenz)"
)
add_result(result_lr)

## 7.3 Modell 1: Einfaches NN (1 Hidden Layer)

Zunächst ein minimales Netz als Ausgangspunkt.

In [ ]:
def build_model(hidden_layers, activation='relu', learning_rate=0.001):
    """Erstelle ein Sequential-Modell mit gegebener Architektur."""
    model = keras.Sequential()
    model.add(layers.Input(shape=(X_train.shape[1],)))
    
def build_model(hidden_layers, activation='relu', learning_rate=0.001):
    """Erstelle ein Sequential-Modell mit gegebener Architektur."""
    model = keras.Sequential()
    model.add(layers.Input(shape=(X_train_nn.shape[1],)))  # besser X_train_nn
    
    for units in hidden_layers:
        model.add(
            layers.Dense(
                units,
                activation=activation,
                kernel_regularizer=keras.regularizers.l2(0.001)
            )
        )
    
    model.add(layers.Dense(1))  # Regression Output
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss='mse',
        metrics=['mae'],
    )
    return model

def train_and_evaluate(model, name, epochs=300, batch_size=32, verbose=0):

    # Early Stopping
    early_stop = keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=20,
        restore_best_weights=True
    )

    # Training
    history = model.fit(
        X_train_nn, y_train_nn,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        verbose=verbose,
        callbacks=[early_stop]
    )

    # Vorhersagen
    y_train_pred = model.predict(X_train_nn, verbose=0).flatten()
    y_test_pred = model.predict(X_test, verbose=0).flatten()

   
    result = evaluate_predictions(
        y_train_nn,     
        y_train_pred,
        y_test,
        y_test_pred,
        name
    )

    add_result(result)

    return history, result

In [ ]:
model_1 = build_model([32], activation='relu', learning_rate=0.001)
model_1.summary()

history_1, result_1 = train_and_evaluate(model_1, "NN [32] ReLU", epochs=300)

In [ ]:
print("Tatsächliche Epochen:", len(history_1.history["loss"])) #sofern die epochen nicht dem maximalen Wert entsprechen wurde overfitting verhidnert 

### 7.3.1 Vergleich mit und ohne L2-Regularisierung

In [ ]:
def build_model_plain(hidden_layers, activation='relu', learning_rate=0.001):
    model = keras.Sequential()
    model.add(layers.Input(shape=(X_train_nn.shape[1],)))
    
    for units in hidden_layers:
        model.add(layers.Dense(units, activation=activation))
    
    model.add(layers.Dense(1))
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss='mse',
        metrics=['mae']
    )
    
    return model


def build_model_l2(hidden_layers, activation='relu', learning_rate=0.001, l2_value=0.001):
    model = keras.Sequential()
    model.add(layers.Input(shape=(X_train_nn.shape[1],)))
    
    for units in hidden_layers:
        model.add(
            layers.Dense(
                units,
                activation=activation,
                kernel_regularizer=keras.regularizers.l2(l2_value)
            )
        )
    
    model.add(layers.Dense(1))
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss='mse',
        metrics=['mae']
    )
    
    return model

Modelle Trainieren

In [ ]:
# Plain Modell
model_plain = build_model_plain([64, 32])
history_plain, result_plain = train_and_evaluate(
    model_plain,
    "NN Plain",
    epochs=300
)

# L2 Modell
model_l2 = build_model_l2([64, 32])
history_l2, result_l2 = train_and_evaluate(
    model_l2,
    "NN L2",
    epochs=300
)

Parameterzahl vergleichen

In [ ]:
print("Plain Parameter:", model_plain.count_params())
print("L2 Parameter:", model_l2.count_params())

Tatsächliche Epochen vergleichen

In [ ]:
print("Plain Epochen:", len(history_plain.history["loss"]))
print("L2 Epochen:", len(history_l2.history["loss"]))

Overfitting differnez berechnen 

In [ ]:
def overfit_gap(result):
    return result["R² Train"] - result["R² Test"]

print("Plain Overfit Gap:", overfit_gap(result_plain))
print("L2 Overfit Gap:", overfit_gap(result_l2))

Lernkurven vergleichen

In [ ]:
plt.figure(figsize=(10,5))

plt.plot(history_plain.history["val_loss"], label="Plain Val Loss")
plt.plot(history_l2.history["val_loss"], label="L2 Val Loss")

plt.xlabel("Epoch")
plt.ylabel("Validation Loss")
plt.title("Validation Loss Comparison")
plt.legend()
plt.show()

Residuenvergleich 

In [ ]:
y_test_pred_plain = model_plain.predict(X_test).flatten()
y_test_pred_l2 = model_l2.predict(X_test).flatten()

res_plain = y_test - y_test_pred_plain
res_l2 = y_test - y_test_pred_l2

plt.figure(figsize=(10,5))
plt.scatter(y_test_pred_plain, res_plain, alpha=0.4, label="Plain")
plt.scatter(y_test_pred_l2, res_l2, alpha=0.4, label="L2")
plt.axhline(0)
plt.legend()
plt.title("Residual Comparison")
plt.show()

#### Vergleich der Modelle mit und ohne L2-Regularisierung:

Das neuronale Netz ohne Regularisierung (Plain) erreicht einen Test-R² von 0.7637 bei einem Overfitting-Gap von ca. 0.040. Das Modell mit L2-Regularisierung erzielt hingegen einen leicht höheren Test-R² von 0.7715 und weist mit ca. 0.024 einen deutlich geringeren Overfitting-Gap auf.

Die L2-Regularisierung reduziert somit die Differenz zwischen Trainings- und Testleistung und verbessert gleichzeitig die Generalisierungsfähigkeit des Modells. Auch die Validierungskurve verläuft stabiler und zeigt weniger Schwankungen als beim unregularisierten Modell.

Die Residuenverteilung beider Modelle ist insgesamt ähnlich, jedoch wirkt das L2-Modell homogener und zeigt eine etwas gleichmäßigere Streuung.

Insgesamt deutet dies darauf hin, dass die L2-Regularisierung Overfitting erfolgreich reduziert und die Modellstabilität erhöht, ohne die Vorhersageleistung zu verschlechtern. Das L2-Modell stellt daher in diesem Vergleich die robustere Variante dar.

### 7.3.2 Vergleich von Plain L2 Dropout und L2 + Dropout 

In [ ]:

# 1) Modellfunktionen


def build_model_plain(hidden_layers, activation='relu', learning_rate=0.001):
    model = keras.Sequential()
    model.add(layers.Input(shape=(X_train_nn.shape[1],)))
    
    for units in hidden_layers:
        model.add(layers.Dense(units, activation=activation))
    
    model.add(layers.Dense(1))
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss='mse',
        metrics=['mae']
    )
    
    return model


def build_model_l2(hidden_layers, activation='relu', learning_rate=0.001, l2_value=0.001):
    model = keras.Sequential()
    model.add(layers.Input(shape=(X_train_nn.shape[1],)))
    
    for units in hidden_layers:
        model.add(
            layers.Dense(
                units,
                activation=activation,
                kernel_regularizer=keras.regularizers.l2(l2_value)
            )
        )
    
    model.add(layers.Dense(1))
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss='mse',
        metrics=['mae']
    )
    
    return model


def build_model_dropout(hidden_layers, activation='relu', learning_rate=0.001, dropout_rate=0.2):
    model = keras.Sequential()
    model.add(layers.Input(shape=(X_train_nn.shape[1],)))
    
    for units in hidden_layers:
        model.add(layers.Dense(units, activation=activation))
        model.add(layers.Dropout(dropout_rate))
    
    model.add(layers.Dense(1))
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss='mse',
        metrics=['mae']
    )
    
    return model


def build_model_l2_dropout(hidden_layers, activation='relu', learning_rate=0.001, l2_value=0.001, dropout_rate=0.2):
    model = keras.Sequential()
    model.add(layers.Input(shape=(X_train_nn.shape[1],)))
    
    for units in hidden_layers:
        model.add(
            layers.Dense(
                units,
                activation=activation,
                kernel_regularizer=keras.regularizers.l2(l2_value)
            )
        )
        model.add(layers.Dropout(dropout_rate))
    
    model.add(layers.Dense(1))
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss='mse',
        metrics=['mae']
    )
    
    return model



# 2) Overfit-Funktion (mit R²)


def overfit_gap(result):
    return result["R² Train"] - result["R² Test"]






In [ ]:

# 3) Modelle trainieren


model_plain = build_model_plain([64, 32])
history_plain, result_plain = train_and_evaluate(model_plain, "NN Plain", epochs=300)

model_l2 = build_model_l2([64, 32])
history_l2, result_l2 = train_and_evaluate(model_l2, "NN L2", epochs=300)

model_dropout = build_model_dropout([64, 32])
history_dropout, result_dropout = train_and_evaluate(model_dropout, "NN Dropout", epochs=300)

model_l2_dropout = build_model_l2_dropout([64, 32])
history_l2_dropout, result_l2_dropout = train_and_evaluate(model_l2_dropout, "NN L2 + Dropout", epochs=300)


In [ ]:

# 4) Vergleichstabelle (mit R²)


model_results = [
    {
        "Modell": "NN Plain",
        "R² Train": result_plain["R² Train"],
        "R² Test": result_plain["R² Test"],
        "Overfit Gap": overfit_gap(result_plain),
        "MAE Test": result_plain["MAE Test"],
        "Epochen": len(history_plain.history["loss"]),
        "Parameter": model_plain.count_params()
    },
    {
        "Modell": "NN L2",
        "R² Train": result_l2["R² Train"],
        "R² Test": result_l2["R² Test"],
        "Overfit Gap": overfit_gap(result_l2),
        "MAE Test": result_l2["MAE Test"],
        "Epochen": len(history_l2.history["loss"]),
        "Parameter": model_l2.count_params()
    },
    {
        "Modell": "NN Dropout",
        "R² Train": result_dropout["R² Train"],
        "R² Test": result_dropout["R² Test"],
        "Overfit Gap": overfit_gap(result_dropout),
        "MAE Test": result_dropout["MAE Test"],
        "Epochen": len(history_dropout.history["loss"]),
        "Parameter": model_dropout.count_params()
    },
    {
        "Modell": "NN L2 + Dropout",
        "R² Train": result_l2_dropout["R² Train"],
        "R² Test": result_l2_dropout["R² Test"],
        "Overfit Gap": overfit_gap(result_l2_dropout),
        "MAE Test": result_l2_dropout["MAE Test"],
        "Epochen": len(history_l2_dropout.history["loss"]),
        "Parameter": model_l2_dropout.count_params()
    }
]

comparison_df = pd.DataFrame(model_results).round(4)

comparison_df

Vergleich der verschiedenen Modellvarianten:

Das neuronale Netz ohne Regularisierung (NN Plain) erzielt mit einem Test-R² von 0.7911 die beste Vorhersageleistung. Allerdings weist es mit einem Overfitting-Gap von 0.0600 die größte Differenz zwischen Trainings- und Testleistung auf.

Die L2-Regularisierung reduziert das Overfitting deutlich (Gap 0.0243), führt jedoch zu einer leicht geringeren Testperformance (R² = 0.7860). Das Dropout-Modell verringert das Overfitting weiter, erreicht jedoch ebenfalls eine geringere Testleistung. Die Kombination aus L2 und Dropout zeigt das geringste Overfitting, allerdings auch die niedrigste Testperformance.

Insgesamt liefert das Plain-Modell die höchste Genauigkeit, während L2-Regularisierung die beste Balance zwischen Performance und Generalisierungsfähigkeit bietet.

## 7.4 Modell 2: Tieferes Netz (3 Hidden Layers)

In [ ]:
model_2 = build_model([64, 32, 16], activation='relu', learning_rate=0.001)
model_2.summary()

history_2, result_2 = train_and_evaluate(model_2, "NN [64,32,16] ReLU", epochs=200)

## 7.5 Modell 3: Breiteres Netz

In [ ]:
model_3 = build_model([128, 64, 32], activation='relu', learning_rate=0.001)
model_3.summary()

history_3, result_3 = train_and_evaluate(model_3, "NN [128,64,32] ReLU", epochs=200)

## 7.6 Modell 4: Verschiedene Aktivierungsfunktionen

In [ ]:
results_activation = {}

for act in ['relu', 'elu', 'tanh', 'sigmoid']:
    model = build_model([64, 32, 16], activation=act, learning_rate=0.001)
    history, result = train_and_evaluate(model, f"NN [64,32,16] {act}", epochs=200)
    results_activation[act] = result

## 7.7 Modell 5: Tiefes Netz mit Regularisierung

In [ ]:
model_5 = keras.Sequential([
    layers.Input(shape=(X_train.shape[1],)),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.1),
    layers.Dense(16, activation='relu'),
    layers.Dense(1),
])

model_5.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='mse',
    metrics=['mae'],
)

model_5.summary()
history_5, result_5 = train_and_evaluate(model_5, "NN [128,64,32,16] + Dropout", epochs=300)

## 7.8 Lernkurven visualisieren

In [ ]:
def plot_training_history(histories, names):
    """Lernkurven mehrerer Modelle."""
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    for hist, name in zip(histories, names):
        axes[0].plot(hist.history['loss'], label=f'{name} (Train)')
        axes[0].plot(hist.history['val_loss'], '--', label=f'{name} (Val)')
    
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('MSE Loss')
    axes[0].set_title('Lernkurven: Loss')
    axes[0].legend(fontsize=8)
    axes[0].set_yscale('log')
    axes[0].grid(True, alpha=0.3)
    
    for hist, name in zip(histories, names):
        axes[1].plot(hist.history['mae'], label=f'{name} (Train)')
        axes[1].plot(hist.history['val_mae'], '--', label=f'{name} (Val)')
    
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('MAE')
    axes[1].set_title('Lernkurven: MAE')
    axes[1].legend(fontsize=8)
    axes[1].grid(True, alpha=0.3)
    
    fig.tight_layout()
    save_fig(fig, 'nn_learning_curves')
    return fig

fig = plot_training_history(
    [history_1, history_2, history_3, history_5],
    ['[32]', '[64,32,16]', '[128,64,32]', '[128,64,32,16]+Drop']
)
plt.show()

## 7.9 Vergleich NN vs. Lineare Regression

In [ ]:
# Bestes NN-Modell vs. Lineare Regression
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

plot_predicted_vs_actual(
    y_test, lr.predict(X_test),
    title="Lineare Regression", ax=axes[0]
)

# Das beste NN-Modell (manuell auswählen nach Ergebnissen)
best_nn = model_5  # Anpassen je nach Ergebnis
y_pred_nn = best_nn.predict(X_test, verbose=0).flatten()
plot_predicted_vs_actual(
    y_test, y_pred_nn,
    title="Neuronales Netz (bestes Modell)", ax=axes[1]
)

fig.suptitle("Kernvergleich: NN vs. Lineare Regression", fontsize=14)
fig.tight_layout()
save_fig(fig, 'nn_vs_linear_regression')
plt.show()

## 7.10 Zusammenfassung

| Aspekt | Lineare Regression | Neuronales Netz |
|--------|-------------------|-----------------|
| R² Test | _eintragen_ | _eintragen_ |
| MAE Test | _eintragen_ | _eintragen_ |
| Trainingszeit | < 1s | _eintragen_ |
| Interpretierbarkeit | Hoch (Koeffizienten) | Gering (Black Box) |
| Hyperparameter | Keine | Architektur, LR, Epochs, ... |

**Fazit:**
- _Hier Ergebnisse und Interpretation eintragen_
- _Wann lohnt sich ein NN gegenüber linearer Regression?_
- _Grenzen und offene Fragen_